In [1]:
# Dependencies
from sqlalchemy import create_engine
from datetime import timedelta, datetime, tzinfo
from zipfile import ZipFile
import collections
#import psycopg2
import requests
import json
import pandas as pd

In [2]:
##Import the csv data

In [3]:
#data frame for legal status and pop
lgpure_df = pd.read_csv("legaldata.csv")
#data frame for mvr deaths
MVR_df = pd.read_csv("mvdeaths.csv")
#data frame for overdose deaths
overdose_df = pd.read_csv("overdose.csv")

In [4]:
####legal status for map

In [5]:
lgpure_df.head(3)

,State,Pop,legalWeedStatus,medicinalWeedStatus,decriminalizedWeedStatus,state
0,Alabama,4908621,Fully Illegal,False,False,Alabama
1,Alaska,734002,Fully Legal,True,True,Alaska
2,Arizona,7378494,Mixed,True,False,Arizona


In [6]:
lgpure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   State                     51 non-null     object
 1   Pop                       51 non-null     int64 
 2   legalWeedStatus           51 non-null     object
 3   medicinalWeedStatus       51 non-null     bool  
 4   decriminalizedWeedStatus  51 non-null     bool  
 5   state                     51 non-null     object
dtypes: bool(2), int64(1), object(3)
memory usage: 1.8+ KB


In [7]:
##df to join to other df (MVR and Overdose)
legal_df = lgpure_df.drop(columns=["state","medicinalWeedStatus","decriminalizedWeedStatus"])
legal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   State            51 non-null     object
 1   Pop              51 non-null     int64 
 2   legalWeedStatus  51 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.3+ KB


In [8]:
###df for legal map


In [9]:
legal_mapdf = legal_df.drop(columns=["Pop"])
legal_mapdf.head()

,State,legalWeedStatus
0,Alabama,Fully Illegal
1,Alaska,Fully Legal
2,Arizona,Mixed
3,Arkansas,Mixed
4,California,Fully Legal


In [10]:
###export to csv
legal_mapdf.to_csv(r'C:\Users\jcaravella\Documents\Project2\Main_data\state_status.csv', index = False,header=True)

In [11]:
###############################################################################################################

In [12]:
####MVR info

In [13]:
MVR_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 7 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Year                                            408 non-null    int64  
 1   State                                           408 non-null    object 
 2   Motor Vehicle Deaths - state of residence       408 non-null    int64  
 3   Traffic deaths - state of incident              408 non-null    int64  
 4   Deaths  per 100,000 population                  408 non-null    float64
 5   Traffic deaths  per 10,000 motor vehicles       408 non-null    float64
 6   Traffic deaths - per 100,000,000 vehicle miles  408 non-null    float64
dtypes: float64(3), int64(3), object(1)
memory usage: 22.4+ KB


In [14]:
##drop to the same years as of od deaths
years = ['2015','2016','2017']
mvr_neededyears =MVR_df[MVR_df.Year.isin(years)]
mvr_neededyears.head()

,Year,State,Motor Vehicle Deaths - state of residence,Traffic deaths - state of incident,"Deaths per 100,000 population","Traffic deaths per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles"
0,2017,Alabama,1034,948,21.2,1.9,1.3
1,2016,Alabama,1158,1122,23.8,2.1,1.6
2,2015,Alabama,962,849,19.8,1.6,1.3
8,2017,Alaska,101,79,13.7,1.0,1.4
9,2016,Alaska,97,81,13.1,1.0,1.5


In [15]:
#####Filtering and joining

In [16]:
##Join legal status
mvr_joined =  mvr_neededyears.join(legal_df.set_index(['State']), on=['State'], how='left')
mvr_joined.head()
#mvr_status = mvr_joined.drop.na()

,Year,State,Motor Vehicle Deaths - state of residence,Traffic deaths - state of incident,"Deaths per 100,000 population","Traffic deaths per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles",Pop,legalWeedStatus
0,2017,Alabama,1034,948,21.2,1.9,1.3,4908621,Fully Illegal
1,2016,Alabama,1158,1122,23.8,2.1,1.6,4908621,Fully Illegal
2,2015,Alabama,962,849,19.8,1.6,1.3,4908621,Fully Illegal
8,2017,Alaska,101,79,13.7,1.0,1.4,734002,Fully Legal
9,2016,Alaska,97,81,13.1,1.0,1.5,734002,Fully Legal


In [17]:
#drop na
mvr_status = mvr_joined.dropna()
mvr_status.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 0 to 402
Data columns (total 9 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Year                                            153 non-null    int64  
 1   State                                           153 non-null    object 
 2   Motor Vehicle Deaths - state of residence       153 non-null    int64  
 3   Traffic deaths - state of incident              153 non-null    int64  
 4   Deaths  per 100,000 population                  153 non-null    float64
 5   Traffic deaths  per 10,000 motor vehicles       153 non-null    float64
 6   Traffic deaths - per 100,000,000 vehicle miles  153 non-null    float64
 7   Pop                                             153 non-null    int64  
 8   legalWeedStatus                                 153 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory us

In [18]:
mvr_status.head(3)

,Year,State,Motor Vehicle Deaths - state of residence,Traffic deaths - state of incident,"Deaths per 100,000 population","Traffic deaths per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles",Pop,legalWeedStatus
0,2017,Alabama,1034,948,21.2,1.9,1.3,4908621,Fully Illegal
1,2016,Alabama,1158,1122,23.8,2.1,1.6,4908621,Fully Illegal
2,2015,Alabama,962,849,19.8,1.6,1.3,4908621,Fully Illegal


In [19]:
###Dropping all columuns I don't need
mvr_state_yr = mvr_status.drop(columns=["Deaths  per 100,000 population","Motor Vehicle Deaths - state of residence","Traffic deaths  per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles","Pop"])
###Rename traffic deaths
mvr_state_yr = mvr_state_yr.rename(columns={"Traffic deaths - state of incident":"MV_Deaths"})
mvr_state_yr.info()  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 0 to 402
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Year             153 non-null    int64 
 1   State            153 non-null    object
 2   MV_Deaths        153 non-null    int64 
 3   legalWeedStatus  153 non-null    object
dtypes: int64(2), object(2)
memory usage: 6.0+ KB


In [20]:
###Sum up the number of deaths
mvr_state = mvr_state_yr.groupby(['State','legalWeedStatus'])['MV_Deaths'].sum().reset_index()
mvr_state.head()

,State,legalWeedStatus,MV_Deaths
0,Alabama,Fully Illegal,2919
1,Alaska,Fully Legal,225
2,Arizona,Mixed,2821
3,Arkansas,Mixed,1608
4,California,Fully Legal,10779


In [21]:
###Sum year and status
mvr_status = mvr_state_yr.groupby(['Year','legalWeedStatus'])['MV_Deaths'].sum().reset_index()
mvr_status.head()

,Year,legalWeedStatus,MV_Deaths
0,2015,Fully Illegal,7262
1,2015,Fully Legal,6632
2,2015,Mixed,21198
3,2016,Fully Illegal,8131
4,2016,Fully Legal,7752


In [22]:
####Overdose info
overdose_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28224 entries, 0 to 28223
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   State                          28224 non-null  object 
 1   Year                           28224 non-null  int64  
 2   Month                          28224 non-null  object 
 3   Period                         28224 non-null  object 
 4   Indicator                      28224 non-null  object 
 5   Data Value                     22892 non-null  float64
 6   Percent Complete               28224 non-null  object 
 7   Percent Pending Investigation  28224 non-null  float64
 8   State Name                     28224 non-null  object 
 9   Footnote                       28224 non-null  object 
 10  Footnote Symbol                28224 non-null  object 
 11  Predicted Value                17345 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usag

In [23]:
####Initial filter####

In [23]:
###filter clean to only columns I need

##od to the same years as mvr
overdose = overdose_df[overdose_df.Year.isin(years)]
#states, indicator, year
od_drop_df_na = overdose.drop(columns=["Period","State","Percent Complete","Percent Pending Investigation","Footnote","Footnote Symbol","Footnote Symbol","Predicted Value"])
#drop nan
od_df = od_drop_df_na.dropna()


###join the status
od_df = od_drop_df_na.join(legal_mapdf.set_index(['State']), on=['State Name'],how='left')
od_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 18144 entries, 0 to 27983
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             18144 non-null  int64  
 1   Month            18144 non-null  object 
 2   Indicator        18144 non-null  object 
 3   Data Value       13702 non-null  float64
 4   State Name       18144 non-null  object 
 5   legalWeedStatus  18144 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 992.2+ KB


In [24]:
###Sum up by month

In [25]:
####First filtor to needed indicators
totaldeaths1 = od_df[od_df['Indicator']=='Number of Deaths']
totaldeaths =  totaldeaths1.drop(columns=['Indicator'])
totaldeaths.dropna()
totaldeaths.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1836 entries, 4 to 27977
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             1836 non-null   int64  
 1   Month            1836 non-null   object 
 2   Data Value       1836 non-null   float64
 3   State Name       1836 non-null   object 
 4   legalWeedStatus  1836 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 86.1+ KB


In [26]:
#Change name of Colum for death counts

In [27]:
#rename column name
totaldeaths = totaldeaths.rename(columns={"Data Value":"Total_Deaths"})

totaldeaths.head()

,Year,Month,Total_Deaths,State Name,legalWeedStatus
4,2015,April,4133.0,Alaska,Fully Legal
16,2015,August,4222.0,Alaska,Fully Legal
30,2015,December,4193.0,Alaska,Fully Legal
45,2015,February,4084.0,Alaska,Fully Legal
49,2015,January,4034.0,Alaska,Fully Legal


In [28]:
##sum by year and state

In [29]:
Deaths_yr_states = totaldeaths.groupby(['State Name','Year'])['Total_Deaths'].sum().reset_index()
Deaths_yr_states.head()

,State Name,Year,Total_Deaths
0,Alabama,2015,607641.0
1,Alabama,2016,608611.0
2,Alabama,2017,623769.0
3,Alaska,2015,50086.0
4,Alaska,2016,51141.0


In [30]:
#sum by state no year

In [31]:
Deaths_states = totaldeaths.groupby(['State Name'])['Total_Deaths'].sum().reset_index()
Deaths_states.head()

,State Name,Total_Deaths
0,Alabama,1840021.0
1,Alaska,152824.0
2,Arizona,2007843.0
3,Arkansas,1112767.0
4,California,9398586.0


In [32]:
#sum by status no state with year

In [34]:
Deaths_status_year = totaldeaths.groupby(['legalWeedStatus','Year'])['Total_Deaths'].sum().reset_index()
Deaths_status_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   legalWeedStatus  9 non-null      object 
 1   Year             9 non-null      int64  
 2   Total_Deaths     9 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 344.0+ bytes


In [33]:
########################################

In [34]:
################OD sum

In [35]:
#Filter to OD
OD1 = od_df[od_df['Indicator']=='Number of Drug Overdose Deaths']
totalod =  OD1.drop(columns=['Indicator'])
totalod.dropna()
totalod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1836 entries, 0 to 27978
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             1836 non-null   int64  
 1   Month            1836 non-null   object 
 2   Data Value       1836 non-null   float64
 3   State Name       1836 non-null   object 
 4   legalWeedStatus  1836 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 86.1+ KB


In [36]:
#rename column name
totalod = totalod.rename(columns={"Data Value":"Total_OD_Deaths"})

totalod.head()

,Year,Month,Total_OD_Deaths,State Name,legalWeedStatus
0,2015,April,126.0,Alaska,Fully Legal
17,2015,August,124.0,Alaska,Fully Legal
25,2015,December,121.0,Alaska,Fully Legal
40,2015,February,127.0,Alaska,Fully Legal
50,2015,January,126.0,Alaska,Fully Legal


In [37]:
#sum by state year and status
od_yr_states = totalod.groupby(['State Name','Year'])['Total_OD_Deaths'].sum().reset_index()
od_yr_states.head()

,State Name,Year,Total_OD_Deaths
0,Alabama,2015,8818.0
1,Alabama,2016,8324.0
2,Alabama,2017,9551.0
3,Alaska,2015,1472.0
4,Alaska,2016,1594.0


In [38]:
##sum by year legal
od_status_year = totalod.groupby(['legalWeedStatus','Year'])['Total_OD_Deaths'].sum().reset_index()
od_status_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   legalWeedStatus  9 non-null      object 
 1   Year             9 non-null      int64  
 2   Total_OD_Deaths  9 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 344.0+ bytes


In [39]:
##sum by state

In [40]:
OD_states = totalod.groupby(['State Name'])['Total_OD_Deaths'].sum().reset_index()
OD_states.head()

,State Name,Total_OD_Deaths
0,Alabama,26693.0
1,Alaska,4637.0
2,Arizona,48545.0
3,Arkansas,13904.0
4,California,171561.0


In [41]:
#########Joins state tables#########

In [42]:
###join all the status info into one table
#status_join1 = status_pop.join(status_mvrtotal.set_index(['legalWeedStatus']),on=['legalWeedStatus'],how='left')
#status_join1.head()

In [43]:
####OD and deaths
Deaths_yr_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   State Name    153 non-null    object 
 1   Year          153 non-null    int64  
 2   Total_Deaths  153 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.7+ KB


In [44]:
mvr_state_yr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 0 to 402
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Year             153 non-null    int64 
 1   State            153 non-null    object
 2   MV_Deaths        153 non-null    int64 
 3   legalWeedStatus  153 non-null    object
dtypes: int64(2), object(2)
memory usage: 6.0+ KB


In [45]:
od_yr_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   State Name       153 non-null    object 
 1   Year             153 non-null    int64  
 2   Total_OD_Deaths  153 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.7+ KB


In [46]:
####State Year

State_year = mvr_state_yr.join(Deaths_yr_states.set_index(['Year','State Name']),on=['Year','State'],how='left')
###Add #OD
State_year_counts = State_year.join(od_yr_states.set_index(['Year','State Name']),on=['Year','State'],how='left')


In [47]:
State_year_counts.head()

,Year,State,MV_Deaths,legalWeedStatus,Total_Deaths,Total_OD_Deaths
0,2017,Alabama,948,Fully Illegal,623769.0,9551.0
1,2016,Alabama,1122,Fully Illegal,608611.0,8324.0
2,2015,Alabama,849,Fully Illegal,607641.0,8818.0
8,2017,Alaska,79,Fully Legal,51597.0,1571.0
9,2016,Alaska,81,Fully Legal,51141.0,1594.0


In [48]:
##sum by year legal
Status_Counts =State_year_counts.groupby(['legalWeedStatus','Year']).sum().reset_index()
Status_Counts.head()


,legalWeedStatus,Year,MV_Deaths,Total_Deaths,Total_OD_Deaths
0,Fully Illegal,2015,7262,5410542.0,89453.0
1,Fully Illegal,2016,8131,5441650.0,99669.0
2,Fully Illegal,2017,7658,5590371.0,114281.0
3,Fully Legal,2015,6632,7034178.0,141214.0
4,Fully Legal,2016,7752,7132055.0,154115.0


In [58]:
##sum by year legal
State_Counts =State_year_counts.groupby(['legalWeedStatus','State']).sum().reset_index()
State_Counts['OverdosePercent'] = State_Counts['Total_OD_Deaths']/State_Counts['Total_Deaths']
State_Counts['MVPercent'] = State_Counts['MV_Deaths']/State_Counts['Total_Deaths']
State_Counts.head()

,legalWeedStatus,State,Year,MV_Deaths,Total_Deaths,Total_OD_Deaths,OverdosePercent,MVPercent
0,Fully Illegal,Alabama,6048,2919,1840021.0,26693.0,0.014507,0.001586
1,Fully Illegal,Idaho,6048,726,475742.0,8000.0,0.016816,0.001526
2,Fully Illegal,Kansas,6048,1231,925159.0,11471.0,0.012399,0.001331
3,Fully Illegal,Kentucky,6048,2349,1685606.0,49178.0,0.029175,0.001394
4,Fully Illegal,Mississippi,6048,2104,1110201.0,11751.0,0.010585,0.001895


In [59]:
State_Counts.to_csv(r'C:\Users\jcaravella\Documents\Project2\Main_data\state_level.csv', index = False,header=True)

In [61]:
##sum by year legal
Status_Counts =State_year_counts.groupby(['legalWeedStatus','Year']).sum().reset_index()
Status_Counts['OverdosePercent'] = Status_Counts['Total_OD_Deaths']/Status_Counts['Total_Deaths']
Status_Counts['MVPercent'] = Status_Counts['MV_Deaths']/Status_Counts['Total_Deaths']
Status_Counts.head()

,legalWeedStatus,Year,MV_Deaths,Total_Deaths,Total_OD_Deaths,OverdosePercent,MVPercent
0,Fully Illegal,2015,7262,5410542.0,89453.0,0.016533,0.001342
1,Fully Illegal,2016,8131,5441650.0,99669.0,0.018316,0.001494
2,Fully Illegal,2017,7658,5590371.0,114281.0,0.020442,0.001370
3,Fully Legal,2015,6632,7034178.0,141214.0,0.020075,0.000943
4,Fully Legal,2016,7752,7132055.0,154115.0,0.021609,0.001087


In [62]:
Status_Counts.to_csv(r'C:\Users\jcaravella\Documents\Project2\Main_data\legalstatus_level.csv', index = False,header=True)

In [ ]:
###od deaths / yearly deaths
state_od_td_yr['%OD Deaths'] = state_od_td_yr['yearly_OD_deaths']/state_od_td_yr['yearly_deaths']

In [ ]:
######Stop####

In [41]:
####to get number of mvr deaths by state, dropping tables when not using year
mvr_state_total_md = mvr_status.drop(columns=["Year","Deaths  per 100,000 population","Motor Vehicle Deaths - state of residence","Traffic deaths  per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles","Pop"])
                   
mvr_state_total_md.info()  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 1 to 155
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   State                               153 non-null    object
 1   Traffic deaths - state of incident  153 non-null    int64 
 2   legalWeedStatus                     153 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.8+ KB


In [20]:
####to get number of mvr deaths by state, dropping tables when using year
mvr_yr_state_total_md = mvr_status.drop(columns=["Deaths  per 100,000 population","Motor Vehicle Deaths - state of residence","Traffic deaths  per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles","Pop"])
total_MVR_by_state_yr = mvr_yr_state_total_md.groupby(['State','legalWeedStatus','Year'])['Traffic deaths - state of incident'].sum().reset_index()                  
total_MVR_by_state_yr.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   State                               153 non-null    object 
 1   legalWeedStatus                     153 non-null    object 
 2   Year                                153 non-null    float64
 3   Traffic deaths - state of incident  153 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 4.9+ KB


In [21]:
###Sum up the number of deaths
total_MVR_by_state = mvr_state_total_md.groupby(['State','legalWeedStatus'])['Traffic deaths - state of incident'].sum().reset_index()
total_MVR_by_state.head()


,State,legalWeedStatus,Traffic deaths - state of incident
0,Alabama,Fully Illegal,2919
1,Alaska,Fully Legal,225
2,Arizona,Mixed,2821
3,Arkansas,Mixed,1608
4,California,Fully Legal,10779


In [22]:
####Mvr state level table
#drop columns I don't need for either state level tabel graph

mvr_state = mvr_status.drop(columns=["Motor Vehicle Deaths - state of residence","Traffic deaths - state of incident","Traffic deaths  per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles","Pop"])
                   
mvr_state.head()                                       


,Year,State,"Deaths per 100,000 population",legalWeedStatus
1,2017.0,Alabama,21.2,Fully Illegal
2,2017.0,Alaska,13.7,Fully Legal
3,2017.0,Arizona,14.5,Mixed
4,2017.0,Arkansas,19.3,Mixed
5,2017.0,California,10.6,Fully Legal


In [23]:
##MVR deaths by legal status

In [24]:
mvr_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 1 to 155
Data columns (total 9 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Year                                            153 non-null    float64
 1   State                                           153 non-null    object 
 2   Motor Vehicle Deaths - state of residence       153 non-null    int64  
 3   Traffic deaths - state of incident              153 non-null    int64  
 4   Deaths  per 100,000 population                  153 non-null    float64
 5   Traffic deaths  per 10,000 motor vehicles       153 non-null    float64
 6   Traffic deaths - per 100,000,000 vehicle miles  153 non-null    float64
 7   Pop                                             153 non-null    float64
 8   legalWeedStatus                                 153 non-null    object 
dtypes: float64(5), int64(2), object(2)
memory us

In [25]:
###drop columns I don't need when including the year

data_legal = mvr_status.drop(columns=["State","Pop","Motor Vehicle Deaths - state of residence","Deaths  per 100,000 population","Traffic deaths  per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles"])
#mvr_bystatus = totaldeaths.groupby(['State','Year','Indicator','State Name'])['Data Value'].sum().reset_index()
data_legal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 1 to 155
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Year                                153 non-null    float64
 1   Traffic deaths - state of incident  153 non-null    int64  
 2   legalWeedStatus                     153 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 4.8+ KB


In [26]:
#sums by legal status 
#pop sum
status_MVR_death = data_legal.groupby(['Year','legalWeedStatus'])['Traffic deaths - state of incident'].sum().reset_index()
status_MVR_death.head()

,Year,legalWeedStatus,Traffic deaths - state of incident
0,2015.0,Fully Illegal,7262
1,2015.0,Fully Legal,6632
2,2015.0,Mixed,21198
3,2016.0,Fully Illegal,8131
4,2016.0,Fully Legal,7752


In [27]:
###get pop by status
status_pop = lgpure_df.groupby(['legalWeedStatus'])['Pop'].sum().reset_index()

In [28]:
status_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   legalWeedStatus  3 non-null      object
 1   Pop              3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


In [29]:
####add pop to MVR deaths

In [30]:
####add pop to MVR deaths
status_pop_mvr =  status_MVR_death.join(status_pop.set_index(['legalWeedStatus']), on=['legalWeedStatus'], how='left')
status_pop_mvr.head()

,Year,legalWeedStatus,Traffic deaths - state of incident,Pop
0,2015.0,Fully Illegal,7262,49127995
1,2015.0,Fully Legal,6632,81471023
2,2015.0,Mixed,21198,200719974
3,2016.0,Fully Illegal,8131,49127995
4,2016.0,Fully Legal,7752,81471023


In [31]:
##MVR Sum
data_legal.head()

,Year,Traffic deaths - state of incident,legalWeedStatus
1,2017.0,948,Fully Illegal
2,2017.0,79,Fully Legal
3,2017.0,1000,Mixed
4,2017.0,493,Mixed
5,2017.0,3602,Fully Legal


In [32]:
#rename column name
d2 = data_legal.rename(columns={"Traffic deaths - state of incident":"Traffic_deaths"})

d2.head()



,Year,Traffic_deaths,legalWeedStatus
1,2017.0,948,Fully Illegal
2,2017.0,79,Fully Legal
3,2017.0,1000,Mixed
4,2017.0,493,Mixed
5,2017.0,3602,Fully Legal


In [33]:
#MVR sums by legal status

status_mvrtotal = d2.groupby(['Year','legalWeedStatus'])['Traffic_deaths'].sum().reset_index()
status_mvrtotal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      float64
 1   legalWeedStatus  9 non-null      object 
 2   Traffic_deaths   9 non-null      int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 344.0+ bytes


In [34]:
#status_mvrtotal.head()

In [35]:
#first join for by status table

In [36]:
status_mvrtotal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      float64
 1   legalWeedStatus  9 non-null      object 
 2   Traffic_deaths   9 non-null      int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 344.0+ bytes


In [37]:
status_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   legalWeedStatus  3 non-null      object
 1   Pop              3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


In [39]:
###join all the status info into one table
status_join1 = status_pop.join(status_mvrtotal.set_index(['legalWeedStatus']),on=['legalWeedStatus'],how='left')
status_join1.head()


,legalWeedStatus,Pop,Year,Traffic_deaths
0,Fully Illegal,49127995,2015.0,7262
0,Fully Illegal,49127995,2016.0,8131
0,Fully Illegal,49127995,2017.0,7658
1,Fully Legal,81471023,2015.0,6632
1,Fully Legal,81471023,2016.0,7752


In [40]:
#status_join1['d_p'] =  status_join1['Traffic_deaths']/status_join1['Pop']
#status_join1.head()

,legalWeedStatus,Pop,Year,Traffic_deaths,d_p
0,Fully Illegal,49127995,2015.0,7262,0.000148
0,Fully Illegal,49127995,2016.0,8131,0.000166
0,Fully Illegal,49127995,2017.0,7658,0.000156
1,Fully Legal,81471023,2015.0,6632,0.000081
1,Fully Legal,81471023,2016.0,7752,0.000095


In [32]:
###Overdose Info###

In [41]:
overdose_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29568 entries, 0 to 29567
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   State                          29568 non-null  object 
 1   Year                           29568 non-null  float64
 2   Month                          29568 non-null  object 
 3   Period                         29568 non-null  object 
 4   Indicator                      29568 non-null  object 
 5   Data Value                     24236 non-null  float64
 6   Percent Complete               29568 non-null  object 
 7   Percent Pending Investigation  29568 non-null  float64
 8   State Name                     29568 non-null  object 
 9   Footnote                       29568 non-null  object 
 10  Footnote Symbol                29568 non-null  object 
 11  Predicted Value                18465 non-null  object 
dtypes: float64(3), object(9)
memory usage: 2.7+ MB

In [43]:
###Total deaths

In [46]:
state_yr_deaths.head()



,State,Year,State Name,legalWeedStatus,yearly_deaths
0,AK,2015.0,Alaska,Fully Legal,50086.0
1,AK,2016.0,Alaska,Fully Legal,51141.0
2,AK,2017.0,Alaska,Fully Legal,51597.0
3,AL,2015.0,Alabama,Fully Illegal,607641.0
4,AL,2016.0,Alabama,Fully Illegal,608611.0


In [47]:
##State years combined
state_deaths = state_yr_deaths.groupby(['State', 'State Name','legalWeedStatus'])['yearly_deaths'].sum().reset_index()
#state_deaths = state_yr_deaths1.rename(columns={"Data Value":"yearly_deaths"})

state_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   State            51 non-null     object 
 1   State Name       51 non-null     object 
 2   legalWeedStatus  51 non-null     object 
 3   yearly_deaths    51 non-null     float64
dtypes: float64(1), object(3)
memory usage: 1.7+ KB


In [40]:
####deaths by legal status

In [39]:
status_deaths = state_yr_deaths.groupby(['Year','legalWeedStatus'])['yearly_deaths'].sum().reset_index()
status_deaths.head()

,Year,legalWeedStatus,yearly_deaths
0,2015.0,Fully Illegal,5410542.0
1,2015.0,Fully Legal,7034178.0
2,2015.0,Mixed,19335816.0
3,2016.0,Fully Illegal,5441650.0
4,2016.0,Fully Legal,7132055.0


In [40]:
status_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      float64
 1   legalWeedStatus  9 non-null      object 
 2   yearly_deaths    9 non-null      float64
dtypes: float64(2), object(1)
memory usage: 344.0+ bytes


In [58]:
####overdose totals

In [59]:
totalod = od_df[od_df['Indicator']=='Number of Drug Overdose Deaths']
totalod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1908 entries, 0 to 29322
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   State            1908 non-null   object 
 1   Year             1908 non-null   float64
 2   Month            1908 non-null   object 
 3   Indicator        1908 non-null   object 
 4   Data Value       1908 non-null   float64
 5   State Name       1908 non-null   object 
 6   legalWeedStatus  1836 non-null   object 
dtypes: float64(2), object(5)
memory usage: 119.2+ KB


In [60]:
state_yr_od1 = totalod.groupby(['Year','legalWeedStatus','State Name'])['Data Value'].sum().reset_index()
state_yr_od = state_yr_od1.rename(columns={"Data Value":"yearly_OD_deaths"})
state_yr_od.head()

,Year,legalWeedStatus,State Name,yearly_OD_deaths
0,2015.0,Fully Illegal,Alabama,8818.0
1,2015.0,Fully Illegal,Idaho,2570.0
2,2015.0,Fully Illegal,Kansas,3926.0
3,2015.0,Fully Illegal,Kentucky,14286.0
4,2015.0,Fully Illegal,Mississippi,3907.0


In [61]:
####state od combined years

state_od1 = totalod.groupby(['legalWeedStatus','State Name'])['Data Value'].sum().reset_index()
state_od = state_od1.rename(columns={"Data Value":"yearly_OD_deaths"})
state_od.head()

,legalWeedStatus,State Name,yearly_OD_deaths
0,Fully Illegal,Alabama,26693.0
1,Fully Illegal,Idaho,8000.0
2,Fully Illegal,Kansas,11471.0
3,Fully Illegal,Kentucky,49178.0
4,Fully Illegal,Mississippi,11751.0


In [62]:
##Join OD deaths to yearly deaths by state

In [63]:
##Join OD deaths to yearly deaths by state
state_od_td_yr = state_yr_deaths.join(state_yr_od.set_index(['Year','State Name','legalWeedStatus']), on = ['Year','State Name','legalWeedStatus'], how='left')
state_od_td_yr.info()     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             153 non-null    object 
 1   Year              153 non-null    float64
 2   State Name        153 non-null    object 
 3   legalWeedStatus   153 non-null    object 
 4   yearly_deaths     153 non-null    float64
 5   yearly_OD_deaths  153 non-null    float64
dtypes: float64(3), object(3)
memory usage: 7.3+ KB


In [71]:
total_MVR_state_year = total_MVR_by_state_yr.rename(columns={"Traffic deaths - state of incident":"MVR_Deaths"})

total_MVR_state_year.head()

,State,legalWeedStatus,Year,MVR_Deaths
0,Alabama,Fully Illegal,2015.0,849
1,Alabama,Fully Illegal,2016.0,1122
2,Alabama,Fully Illegal,2017.0,948
3,Alaska,Fully Legal,2015.0,65
4,Alaska,Fully Legal,2016.0,81


In [72]:
state_od_td_yr = state_od_td_yr.join(total_MVR_state_year.set_index(['Year','State','legalWeedStatus']), on = ['Year','State','legalWeedStatus'], how='left')
state_od_td_yr.info()     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   State                               153 non-null    object 
 1   Year                                153 non-null    float64
 2   State Name                          153 non-null    object 
 3   legalWeedStatus                     153 non-null    object 
 4   yearly_deaths                       153 non-null    float64
 5   yearly_OD_deaths                    153 non-null    float64
 6   Traffic deaths - state of incident  0 non-null      float64
 7   MVR_Deaths                          0 non-null      float64
dtypes: float64(5), object(3)
memory usage: 9.7+ KB


In [ ]:
#add yearly MVR deaths
#then /0d /mvr/deaths
#push by state
#then o without year

In [ ]:
###State totals with and w/out state

In [55]:
####add MVRdeaths
state_year_totals = state_od_td_yr.join(total_MVR_by_state_yr.set_index(['Year','State','legalWeedStatus']), on = ['Year','State','legalWeedStatus'], how='left')
state_year_totals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   State                               153 non-null    object 
 1   Year                                153 non-null    float64
 2   State Name                          153 non-null    object 
 3   legalWeedStatus                     153 non-null    object 
 4   yearly_deaths                       153 non-null    float64
 5   yearly_OD_deaths                    153 non-null    float64
 6   Traffic deaths - state of incident  0 non-null      float64
dtypes: float64(4), object(3)
memory usage: 8.5+ KB


In [48]:
###od deaths / yearly deaths
state_od_td_yr['%OD Deaths'] = state_od_td_yr['yearly_OD_deaths']/state_od_td_yr['yearly_deaths']

In [49]:
state_od_td_yr.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             153 non-null    object 
 1   Year              153 non-null    float64
 2   State Name        153 non-null    object 
 3   legalWeedStatus   153 non-null    object 
 4   yearly_deaths     153 non-null    float64
 5   yearly_OD_deaths  153 non-null    float64
 6   %OD Deaths        153 non-null    float64
dtypes: float64(4), object(3)
memory usage: 8.5+ KB


In [50]:
mvr_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 1 to 155
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Year                            153 non-null    float64
 1   State                           153 non-null    object 
 2   Deaths  per 100,000 population  153 non-null    float64
 3   legalWeedStatus                 153 non-null    object 
dtypes: float64(2), object(2)
memory usage: 6.0+ KB


In [52]:
##Join MVR deaths to yearly deaths by state
state_total_yr = state_od_td_yr.join(mvr_state.set_index(['Year','State','legalWeedStatus']), on = ['Year','State Name','legalWeedStatus'], how='left')
state_total_yr.info()     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   State                           153 non-null    object 
 1   Year                            153 non-null    float64
 2   State Name                      153 non-null    object 
 3   legalWeedStatus                 153 non-null    object 
 4   yearly_deaths                   153 non-null    float64
 5   yearly_OD_deaths                153 non-null    float64
 6   %OD Deaths                      153 non-null    float64
 7   Deaths  per 100,000 population  153 non-null    float64
dtypes: float64(5), object(3)
memory usage: 9.7+ KB


In [53]:
###export to csv
state_total_yr.to_csv(r'C:\Users\jcaravella\Documents\Project2\Main_data\state_total_yr.csv', index = False,header=True)

In [55]:
state_total_yr.head()

,State,Year,State Name,legalWeedStatus,yearly_deaths,yearly_OD_deaths,%OD Deaths,"Deaths per 100,000 population"
0,AK,2015.0,Alaska,Fully Legal,50086.0,1472.0,0.029389,10.0
1,AK,2016.0,Alaska,Fully Legal,51141.0,1594.0,0.031169,13.1
2,AK,2017.0,Alaska,Fully Legal,51597.0,1571.0,0.030448,13.7
3,AL,2015.0,Alabama,Fully Illegal,607641.0,8818.0,0.014512,19.8
4,AL,2016.0,Alabama,Fully Illegal,608611.0,8324.0,0.013677,23.8


In [ ]:
#####od's by legal status####

In [54]:
#drop state
#legal_deaths = state_yr_deaths.drop(columns=["State","State Name"])
####sum by legal type
OD_by_legal_type = state_yr_od.groupby(['Year','legalWeedStatus'])['yearly_OD_deaths'].sum().reset_index()
OD_by_legal_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              9 non-null      int64  
 1   legalWeedStatus   9 non-null      object 
 2   yearly_OD_deaths  9 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 344.0+ bytes


In [57]:
#joins the two together into one
Od_deaths_status_combined = OD_by_legal_type.join(status_deaths.set_index(['Year','legalWeedStatus']), on = ['Year','legalWeedStatus'], how='left')
Od_deaths_status_combined.info()                                                 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              9 non-null      int64  
 1   legalWeedStatus   9 non-null      object 
 2   yearly_OD_deaths  9 non-null      float64
 3   yearly_deaths     9 non-null      float64
dtypes: float64(2), int64(1), object(1)
memory usage: 416.0+ bytes


In [58]:
Od_deaths_status_combined['Year'] = Od_deaths_status_combined['Year'].astype(str)

In [59]:
Od_deaths_status_combined.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              9 non-null      object 
 1   legalWeedStatus   9 non-null      object 
 2   yearly_OD_deaths  9 non-null      float64
 3   yearly_deaths     9 non-null      float64
dtypes: float64(2), object(2)
memory usage: 416.0+ bytes


In [66]:
od_per_death = Od_deaths_status_combined['yearly_OD_deaths']/Od_deaths_status_combined['yearly_deaths']
od_per_death

0    0.016533
1    0.020075
2    0.018626
3    0.018316
4    0.021609
5    0.022112
6    0.020442
7    0.022897
8    0.026359
dtype: float64

In [ ]:
###by status table

In [45]:
status_join1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      object 
 1   legalWeedStatus  9 non-null      object 
 2   Pop              9 non-null      float64
 3   Traffic_deaths   9 non-null      float64
dtypes: float64(2), object(2)
memory usage: 416.0+ bytes


In [60]:
total_joined_status = status_join1.join(Od_deaths_status_combined.set_index(['Year','legalWeedStatus']), on = ['Year','legalWeedStatus'], how='left')
total_joined_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              9 non-null      object 
 1   legalWeedStatus   9 non-null      object 
 2   Pop               9 non-null      float64
 3   Traffic_deaths    9 non-null      float64
 4   yearly_OD_deaths  9 non-null      float64
 5   yearly_deaths     9 non-null      float64
dtypes: float64(4), object(2)
memory usage: 560.0+ bytes


In [62]:
OD_type_status = [['Year', total_joined_status.Year]]
OD_type_status.head()

AttributeError: 'list' object has no attribute 'head'

In [ ]:
##into postgres

In [17]:
#rds_connection_string = f"{dbuser}<insert user name>:<insert password>@localhost:5432/customer_db"
#engine = create_engine(f'postgresql://{rds_connection_string}')
password = "Hooch#12"
dbuser = "postgres"
rds_connection_string = f"{dbuser}:{password}@localhost:5432/project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
###postgresSQL
##push to server

In [ ]:
engine.table_names()

In [ ]:
#pushing Cluster,Bar,Line DATAFRAME to SQL
years_trend.to_sql(name='guns_year', con=engine, if_exists='replace', index=False)
deaths_2018_unindexed.to_sql(name='killings_injuries_2018', con=engine, if_exists='replace', index=False)

In [ ]:

#Creating CSV's of Cluster,Bar,Line DATAFRAME
years_trend.to_csv('year_trend_gun.csv')
deaths_2018.to_csv('deaths_2018.csv')
gun14_monthly.to_csv("gun2014onwrd.csv")

In [26]:
status_pop.info()

NameError: name 'status_pop' is not defined